#### Word合并

In [5]:
from docx import Document
from natsort import natsorted
import os

folder = r"C:\Users\lenonvo\Desktop\code\bbs_spider\WuShu_Archive"
output = os.path.join(folder, "瀚海星云_WuShu_全集合并.docx")

files = [f for f in os.listdir(folder) if f.endswith(".docx") and not f.startswith("WuShu_全集")]
files = natsorted(files)

master = Document()

def is_paragraph_empty(p):
    """判断一个段落是否是真空（只有空格/回车）"""
    return not p.text.strip()

for idx, file in enumerate(files):
    doc = Document(os.path.join(folder, file))

    # 拷贝段落（跳过空段落）
    for p in doc.paragraphs:
        if not is_paragraph_empty(p):
            new_p = master.add_paragraph(p.text)
            new_p.style = p.style

    # 只有当后面还有文件时才分页
    if idx < len(files) - 1:
        master.add_page_break()

# -------- 第二阶段：清理尾部多余空页 --------

def remove_trailing_empty_paragraphs(doc):
    body = doc._body._body
    while len(body) > 0:
        last = body[-1]
        if last.tag.endswith("p"):
            text = "".join(t.text for t in last.iter() if t.text)
            if not text.strip():
                body.remove(last)
            else:
                break
        else:
            break

remove_trailing_empty_paragraphs(master)

master.save(output)

print("合并完成（已清理空白页）：", output)


合并完成（已清理空白页）： C:\Users\lenonvo\Desktop\code\bbs_spider\WuShu_Archive\瀚海星云_WuShu_全集合并.docx


#### Word拆分

In [34]:
import os
import copy
from docx import Document

# ===== 你的路径（按需改）=====
WORD_INPUT  = r"C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\BBS_WuShu_全集合并_带图片.docx"
OUT_DIR     = r"C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\split_100"
POSTS_PER_FILE = 100

# 是否跳过前5篇（True=前5篇单独留在第1个文件开头且不计入100；False=前5也计入100）
SKIP_FIRST_5_NOT_COUNT = False
# =============================

def is_heading1(paragraph):
    try:
        return paragraph.style and paragraph.style.name == "Heading 1"
    except Exception:
        return False

def clone_block_to_doc(dst_doc, block):
    """
    block: paragraph 或 table 的底层 XML element
    直接深拷贝到新文档，保留样式/图片/表格等。
    """
    dst_doc._body._element.append(copy.deepcopy(block))

def iter_block_items(doc):
    """
    以“段落/表格”为单位遍历（保持原文顺序）。
    """
    body = doc.element.body
    for child in body.iterchildren():
        yield child  # 可能是 w:p 或 w:tbl 等

def main():
    os.makedirs(OUT_DIR, exist_ok=True)

    src = Document(WORD_INPUT)

    # 先找出每个帖子（Heading1）在“段落序列”里的索引，用于计数
    # 但真正拆分用的是 block（段落/表格）级别复制，更稳
    paras = src.paragraphs
    h1_para_indexes = [i for i, p in enumerate(paras) if is_heading1(p)]

    if not h1_para_indexes:
        raise RuntimeError("未检测到 Heading 1（帖子标题）。请确认帖子标题样式是否为 Heading 1。")

    print(f"检测到帖子数（Heading 1）：{len(h1_para_indexes)}")

    # 遍历 block，遇到 Heading1 就认为进入新帖子
    part_idx = 1
    post_count_in_part = 0
    total_posts_written = 0
    first5_remaining = 5

    out_doc = Document()
    # 删除新建文档的默认空段
    if out_doc.paragraphs:
        p0 = out_doc.paragraphs[0]._element
        p0.getparent().remove(p0)

    def save_part(doc_to_save, idx):
        out_path = os.path.join(OUT_DIR, f"Part_{idx:03d}.docx")
        doc_to_save.save(out_path)
        print(f"✅ 保存：{out_path}")

    # 用一个小函数判断“当前 block 是否是 Heading1 段落”
    # 因为我们在 block 层复制，需要在源文档里用 XPath 判断该 block 的 style
    def block_is_heading1(block):
        # w:p 才可能是标题段
        if block.tag.endswith('}p') is False:
            return False
        # 查 pStyle
        pPr = block.find(".//{http://schemas.openxmlformats.org/wordprocessingml/2006/main}pPr")
        if pPr is None:
            return False
        pStyle = pPr.find(".//{http://schemas.openxmlformats.org/wordprocessingml/2006/main}pStyle")
        if pStyle is None:
            return False
        val = pStyle.get("{http://schemas.openxmlformats.org/wordprocessingml/2006/main}val")
        # Word 里 Heading 1 的内部样式名通常是 "Heading1"
        return val == "Heading1"

    for block in iter_block_items(src):
        if block_is_heading1(block):
            # 遇到新帖标题 → 计数逻辑
            if SKIP_FIRST_5_NOT_COUNT and first5_remaining > 0:
                first5_remaining -= 1
            else:
                # 若已经满 100 帖，则先存档，再开新文件
                if post_count_in_part >= POSTS_PER_FILE:
                    save_part(out_doc, part_idx)
                    part_idx += 1
                    post_count_in_part = 0

                    out_doc = Document()
                    if out_doc.paragraphs:
                        p0 = out_doc.paragraphs[0]._element
                        p0.getparent().remove(p0)

                post_count_in_part += 1
                total_posts_written += 1

        # 复制当前 block 到输出文档
        clone_block_to_doc(out_doc, block)

    # 保存最后一个 part
    save_part(out_doc, part_idx)

    print("\n==== 完成 ====")
    print(f"输出目录：{OUT_DIR}")
    print(f"共输出文件数：{part_idx}")
    if SKIP_FIRST_5_NOT_COUNT:
        print(f"计入拆分的帖子数（不含前5）：{total_posts_written}")
    else:
        print(f"计入拆分的帖子数（含前5）：{total_posts_written}")

if __name__ == "__main__":
    main()

检测到帖子数（Heading 1）：7944
✅ 保存：C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\split_100\Part_001.docx
✅ 保存：C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\split_100\Part_002.docx
✅ 保存：C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\split_100\Part_003.docx
✅ 保存：C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\split_100\Part_004.docx
✅ 保存：C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\split_100\Part_005.docx
✅ 保存：C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\split_100\Part_006.docx
✅ 保存：C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\split_100\Part_007.docx
✅ 保存：C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\split_100\Part_008.docx
✅ 保存：C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\split_100\Part_009.docx
✅ 保存：C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\split_100\Part_010.docx
✅ 保存：C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\split_100\P

#### 标题加编号

In [30]:
import re
from docx import Document

# ===== 路径配置：改成你的 =====
WORD_INPUT  = r"C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\BBS_WuShu_全集合并.docx"
WORD_OUTPUT = r"C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\BBS_WuShu_全集合并_编号版.docx"

SKIP_FIRST = 5   # 前 5 个帖子不编号
# ==============================

def set_paragraph_text_keep_style(paragraph, new_text: str):
    """
    python-docx 直接 paragraph.text=... 会丢 run 样式。
    这里采取：保留第一个 run 的样式，把其余 run 清空，再写入新文本。
    """
    if not paragraph.runs:
        paragraph.add_run(new_text)
        return

    # 取第一个 run 来承载新文本（保留其样式）
    paragraph.runs[0].text = new_text
    # 其余 run 清空
    for r in paragraph.runs[1:]:
        r.text = ""

def main():
    doc = Document(WORD_INPUT)

    # 收集所有 Heading 1 段落（帖子标题）
    h1_paras = [p for p in doc.paragraphs if p.style and p.style.name == "Heading 1"]

    print(f"检测到 Heading 1 帖子标题数量：{len(h1_paras)}")
    print(f"前 {SKIP_FIRST} 个不编号，其余从 1 开始编号。")

    counter = 1
    for idx, p in enumerate(h1_paras):
        title = p.text.strip()
        if not title:
            continue

        # 前 SKIP_FIRST 个跳过
        if idx < SKIP_FIRST:
            continue

        new_title = f"{counter} {title}"
        set_paragraph_text_keep_style(p, new_title)
        counter += 1

    doc.save(WORD_OUTPUT)
    print(f"✅ 完成：共编号 {counter-1} 个帖子")
    print(f"💾 输出文件：{WORD_OUTPUT}")

if __name__ == "__main__":
    main()

检测到 Heading 1 帖子标题数量：7944
前 5 个不编号，其余从 1 开始编号。
✅ 完成：共编号 7939 个帖子
💾 输出文件：C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\BBS_WuShu_全集合并_编号版.docx


#### 重新编号

In [37]:
import re
from docx import Document

WORD_INPUT  = r"C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\BBS_WuShu_全集合并_编号版.docx"
WORD_OUTPUT = r"C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\BBS_WuShu_全集合并_重新编号.docx"

SKIP_FIRST = 5  # 前5个不编号

# 常见“旧编号前缀”清理：  1 标题 / 1. 标题 / 1、标题 / (1)标题 / （1）标题 / 【1】标题
OLD_PREFIX_RE = re.compile(
    r"^\s*(?:"
    r"(\d{1,6})\s*[\.、]\s*|"      # 123. / 123、
    r"(\d{1,6})\s+|"              # 123 空格
    r"[\(\（]\s*(\d{1,6})\s*[\)\）]\s*|"  # (123) / （123）
    r"[【\[]\s*(\d{1,6})\s*[】\]]\s*"     # 【123】 / [123]
    r")"
)

def strip_old_number_prefix(title: str) -> str:
    if not title:
        return title
    return re.sub(OLD_PREFIX_RE, "", title).strip()

def main():
    doc = Document(WORD_INPUT)

    # 收集所有 Heading 1
    headings = [p for p in doc.paragraphs if p.style and p.style.name == "Heading 1"]
    total = len(headings)

    print(f"检测到 Heading 1 帖子标题数量：{total}")
    print(f"前 {SKIP_FIRST} 个不编号，其余从 1 开始重新编号。")

    renumbered = 0
    num = 1

    for idx, p in enumerate(headings, start=1):
        raw = (p.text or "").strip()
        if not raw:
            continue

        if idx <= SKIP_FIRST:
            # 前5个不编号：但也建议把旧编号去掉吗？
            # 如果你希望前5个保持原样，就注释掉下面两行
            cleaned = strip_old_number_prefix(raw)
            if cleaned != raw:
                p.text = cleaned
            continue

        # 其他：清旧编号 -> 加新编号
        cleaned = strip_old_number_prefix(raw)
        new_title = f"{num} {cleaned}"
        if new_title != raw:
            p.text = new_title
        renumbered += 1
        num += 1

    doc.save(WORD_OUTPUT)
    print(f"✅ 完成：共重新编号 {renumbered} 个帖子")
    print(f"💾 输出文件：{WORD_OUTPUT}")

if __name__ == "__main__":
    main()

检测到 Heading 1 帖子标题数量：7946
前 5 个不编号，其余从 1 开始重新编号。
✅ 完成：共重新编号 7941 个帖子
💾 输出文件：C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\BBS_WuShu_全集合并_重新编号.docx


#### 图片插入Word

In [23]:
import os
from docx import Document
from docx.shared import Inches

from PIL import Image, ImageOps

WORD_INPUT = r"C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\BBS_WuShu_全集合并.docx"
WORD_OUTPUT = r"C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\BBS_WuShu_全集合并_带图片_修复版.docx"
IMAGE_DIR = r"C:\Users\lenonvo\Desktop\code\瀚海星云BBS资料_WuShu\BBS_WuShu_data\images_all"

# 清洗后的图片缓存目录（自动生成）
CLEAN_DIR = os.path.join(os.path.dirname(IMAGE_DIR), "images_clean_png")
os.makedirs(CLEAN_DIR, exist_ok=True)

IMAGE_WIDTH = Inches(5.5)

def is_source_line(text: str) -> bool:
    t = (text or "").strip()
    return (
        t == "--"
        or "※ 来源" in t
        or "瀚海星云" in t
        or "bbs.ustc.edu.cn" in t
    )

def clean_to_png(src_path: str) -> str | None:
    """
    把任意图片清洗成标准 PNG（去 EXIF / 统一色彩 / 修正旋转），返回清洗后路径。
    失败返回 None（会跳过该图）。
    """
    base = os.path.basename(src_path)
    root, _ = os.path.splitext(base)
    out_path = os.path.join(CLEAN_DIR, root + ".png")

    if os.path.exists(out_path):
        return out_path

    try:
        with Image.open(src_path) as im:
            # 修正 EXIF 方向（很多手机图靠它旋转）
            im = ImageOps.exif_transpose(im)

            # 统一成 RGB，避免奇怪的模式/透明通道导致 Word 显示异常
            if im.mode not in ("RGB", "L"):
                im = im.convert("RGB")

            # 可选：太大的图缩一下，避免 Word 体积爆炸
            max_w = 2000
            if im.width > max_w:
                new_h = int(im.height * max_w / im.width)
                im = im.resize((max_w, new_h))

            im.save(out_path, format="PNG", optimize=True)
        return out_path
    except Exception as e:
        print(f"⚠️ 清洗失败，跳过：{src_path} | {e}")
        return None

def collect_images_by_title(image_dir: str):
    """
    图片命名规则：标题_后缀.xxx
    返回：{标题: [img1, img2, ...]}
    """
    mapping = {}
    for fname in os.listdir(image_dir):
        low = fname.lower()
        if not low.endswith((".jpg", ".jpeg", ".png", ".bmp", ".gif", ".webp", ".tif", ".tiff")):
            continue
        if "_" not in fname:
            continue

        title = fname.split("_", 1)[0]
        full_path = os.path.join(image_dir, fname)
        mapping.setdefault(title, []).append(full_path)

    for k in mapping:
        mapping[k].sort()
    return mapping

def main():
    doc = Document(WORD_INPUT)
    paras = doc.paragraphs

    image_map = collect_images_by_title(IMAGE_DIR)
    print(f"🖼️ 发现 {len(image_map)} 个【有图片的帖子标题】")

    inserted_images = set()   # 确保每张图最多插入一次（按文件名/路径去重）
    done_titles = set()       # 确保同标题只处理一次（防止标题重复导致重复插）

    total_inserted = 0
    i = 0
    while i < len(paras):
        p = paras[i]

        if p.style.name != "Heading 1":
            i += 1
            continue

        title = (p.text or "").strip()
        if not title:
            i += 1
            continue

        if title in done_titles:
            i += 1
            continue

        if title not in image_map:
            i += 1
            continue

        images = image_map[title]
        # 找插入点：本帖内第一个来源段（-- / ※来源 / bbs.ustc.edu.cn）
        insert_idx = None
        j = i + 1
        while j < len(paras):
            if paras[j].style.name == "Heading 1":
                break
            if is_source_line(paras[j].text):
                insert_idx = j
                break
            j += 1
        if insert_idx is None:
            insert_idx = j  # 找不到来源段就插在标题到下个标题之前

        print(f"\n📌 帖子：《{title}》候选图片 {len(images)} 张，插入点段落 index={insert_idx}")

        # 在 insert_idx 之前插入图片
        for img_path in images:
            # 每张图最多插入一次
            key = os.path.abspath(img_path)
            if key in inserted_images:
                continue

            cleaned = clean_to_png(img_path)
            if not cleaned:
                continue

            try:
                img_p = doc.add_paragraph()
                run = img_p.add_run()
                run.add_picture(cleaned, width=IMAGE_WIDTH)

                # 把新段落挪到 insert_idx 前
                paras[insert_idx]._element.addprevious(img_p._element)

                inserted_images.add(key)
                total_inserted += 1
                insert_idx += 1
            except Exception as e:
                print(f"⚠️ 插入失败：{img_path} | {e}")

        done_titles.add(title)
        i = insert_idx

    doc.save(WORD_OUTPUT)
    print(f"\n✅ 完成：共插入 {total_inserted} 张图片")
    print(f"💾 输出文件：{WORD_OUTPUT}")
    print(f"🧼 清洗后的PNG缓存目录：{CLEAN_DIR}")

if __name__ == "__main__":
    main()

🖼️ 发现 220 个【有图片的帖子标题】

📌 帖子：《自我介绍》准备插入 1 张图片（去重后）
   ✅ 实际插入 1 张

📌 帖子：《处女图片贴！》准备插入 1 张图片（去重后）
   ✅ 实际插入 1 张

📌 帖子：《跆拳道WTF品式太极八章图示》准备插入 8 张图片（去重后）
   ✅ 实际插入 8 张

📌 帖子：《ＩＴＦ考级》准备插入 1 张图片（去重后）
   ✅ 实际插入 1 张

📌 帖子：《ITF基本步伐》准备插入 1 张图片（去重后）
   ✅ 实际插入 1 张

📌 帖子：《ITF套路入门：3步对打》准备插入 2 张图片（去重后）
   ✅ 实际插入 2 张

📌 帖子：《TKD拉韧带示意图》准备插入 7 张图片（去重后）
   ✅ 实际插入 7 张

📌 帖子：《ITF色带的9个特尔》准备插入 10 张图片（去重后）
   ✅ 实际插入 10 张

📌 帖子：《TKD之歌！》准备插入 1 张图片（去重后）
   ✅ 实际插入 1 张

📌 帖子：《几种练手上功夫的方法！》准备插入 2 张图片（去重后）
   ✅ 实际插入 2 张

📌 帖子：《干枝五势梅花桩-基本功》准备插入 1 张图片（去重后）
   ✅ 实际插入 1 张

📌 帖子：《ＰＳ的2张～《在源道馆成长》》准备插入 2 张图片（去重后）
   ✅ 实际插入 2 张

📌 帖子：《再贴一张。。。》准备插入 1 张图片（去重后）
   ✅ 实际插入 1 张

📌 帖子：《这是什么？》准备插入 2 张图片（去重后）
   ✅ 实际插入 2 张

📌 帖子：《人体穴位图？》准备插入 2 张图片（去重后）
   ✅ 实际插入 2 张

📌 帖子：《叠道服，系道带》准备插入 4 张图片（去重后）
   ✅ 实际插入 4 张

📌 帖子：《三步对打》准备插入 2 张图片（去重后）
   ✅ 实际插入 2 张

📌 帖子：《四周冲拳》准备插入 5 张图片（去重后）
   ✅ 实际插入 5 张

📌 帖子：《天地特尔》准备插入 5 张图片（去重后）
   ✅ 实际插入 5 张

📌 帖子：《岛山特尔》准备插入 8 张图片（去重后）
   ✅ 实际插入 8 张

📌 帖子：《退溪特尔》准备插入 7 张图片（去重后）
   ✅ 实际插入 7 张

📌 帖子：《今晚太极课的照片！》准备插入 13